In [1]:
import urllib
import csv

def get_files_from_pid(pid_url):
    """
        Appends well known magic to the `pid_url` and retrieves the  metadata instead of the 
        landing page. The metadata are parsed to get the mimetype and url.
    """
    import xml.etree.ElementTree as ET
    pid_metadata_url = pid_url + "@format=cmdi"
    ns = "{http://www.clarin.eu/cmd/}"
    metadata = urllib.urlopen(pid_metadata_url).read()
    root = ET.fromstring(metadata)
    proxies = root.findall('.//%sResourceProxy' % ns)
    bitstream_elems = [x for x in proxies if x.attrib.get("id", "").startswith("_")]
    info = [(
        x.find("./%sResourceType" % ns).attrib.get("mimetype", "unknown"),
        x.find("./%sResourceRef" % ns).text
    ) for x in bitstream_elems]
    return info

def plot(arr):
    import pandas as pd
    from matplotlib import pyplot as plt
    from math import radians

    ax = plt.subplot(111, polar=True)
    ax.scatter(x=[radians(float(deg)) for _1, deg in arr], y=[1] * len(arr))
    ax.set_theta_zero_location('N')
    ax.set_theta_direction(-1)

pid_url = "http://hdl.handle.net/11346/TEST--HGGA"
for bitstream_mimetype, bitstream_url in  get_files_from_pid(pid_url):
    print bitstream_url
    data_csv = csv.reader(urllib.urlopen(bitstream_url))
    data_csv = [ [x.strip() for x in line] for line in data_csv ]
    print "Number of rows (with header): %8d" % len(data_csv)
    for i, row in enumerate(data_csv):
        print row
        if i > 5:
            break

# plot(data_csv[1:3000])







https://rda-summerschool.csc.fi/repository/xmlui/bitstream/handle/1/4/KUM_Tower_WDIR_32m_1min_2015.csv?sequence=1
Number of rows (with header):   525601
['samptime(UTC+2)', 'Tower_WDIR_32m']
['2015-01-01 00:00:00', '285.2594']
['2015-01-01 00:01:00', '285.7347']
['2015-01-01 00:02:00', '287.8657']
['2015-01-01 00:03:00', '286.0261']
['2015-01-01 00:04:00', '286.7316']
['2015-01-01 00:05:00', '287.7657']
